In [ ]:
%matplotlib notebook
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation

from IPython.display import YouTubeVideo, HTML, Audio
from bokeh.layouts import column, row
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.plotting import Figure, show, output_notebook
output_notebook()

# Estimadores adaptivos parte I



Hasta ahora hemos estudiando sistemas lineales donde: 
- sus coeficientes quedan fijos luego del diseño y son constantes en el tiempo
- hacen supuestos sobre los estadísticos de la señal/ruido

Qué hacer si
- no podemos hacer supuestos sobre los estadísticos
- los estadísticos de la señal/ruido cambian en el tiempo: **señales no estacionarias**
- estamos en un escenario donde los datos llegan continuamente: data streaming

En estos casos necesitamos un estimador de tipo **adaptivo**, es decir sistemas y filtros cuyos coeficientes se pueden **adaptar** a medida que llegan nuevos datos

Estos estimadores se diseñan de acuerdo a un método de optimización *online*


La siguiente figura muestra algunos ejemplos de aplicaciones de sistemas adaptivos

<img src="../images/adaptive-systems1.png" width="700">

## Gradiente descendente

Sea un vector de pesos $w$ de largo $L+1$ que guarda los coeficientes de un filtro

Sea ahora una función de costo que mapea el vector de pesos a un número real: $J(w): \mathbb{R}^{L+1} \to \mathbb{R}$. La función de costo debe ser tal que a menor $J$ menor sea el error y por ende mejor sea nuestro filtro

Para entrenar un filtro adaptivo 

1. Partimos de una solución inicial $w_0$
1. Modificamos iterativamente $w$ tal que $J(w_{t+1}) < J(w_t)$
1. Nos detenemos al cumplir un cierto criterio 


Una forma de lograr que es relativamente eficiente es la regla del **gradiente descendente** (GD)

$$
w_{t+1} = w_t - \mu \frac{dJ(w)}{dw},
$$

donde $\mu$ se conoce como tasa de aprendizaje o "paso"

- Imaginemos $J$ como una superficie de $L+1$ dimensiones
- En cada punto el gradiente negativo de $J$ nos indica hacia donde está el descenso más abrupto
- La tasa $\mu$ nos da el largo del salto entre $w_t$ y $w_{t+1}$

Observando la **expansión de Taylor de primer orden** de $J$ en $w_{t}$ 

$$
\begin{align}
J(w_{t+1}) &= J(w_t) + \frac{dJ(w_t)}{dw} (w_{t+1} - w_{t})  \nonumber \\
&= J(w_t) -\mu \left \| \frac{dJ(w_t)}{dw} \right \|^2 \leq J(w_t) \nonumber 
\end{align}
$$

es decir que usando la regla GD con $\mu>0$ y asumiendo que $J$ es convexo entonces se cumple que $J$ siempre decrece monotonicamente


### Ejemplo gráfico

La siguiente animación muestra una superficie no convexa de costo con un parámetro unidimensional 

Cada punto representa una solución que parte desde una posición inicial distinta

Observe como evolucionan las tres soluciones con $\mu=0.05$

In [ ]:
J = lambda w : (w-1)**2 + 0.2*np.sin(2*np.pi*w) #  Función de costo
gradJ = lambda w : 2*(w-1) + 0.2*2*np.pi*np.cos(2*np.pi*w) # Gradiente
w = np.linspace(0, 2, num=100)

In [ ]:
%%capture 

mu = 0.05 # Tasa de aprendizaje
wt = np.array([0.05, 0.4, 1.9]) # Solución inicial

fig, ax = plt.subplots(1, figsize=(7, 3), tight_layout=True)

def init():
    global wt
    ax.cla()
    ax.quiver(wt, J(wt), -mu*gradJ(wt), 0.0, scale=1, 
              scale_units='x', width=0.005)
    ax.plot(w, J(w), lw=2, alpha=.5)
    ax.scatter(wt, J(wt), c='k', s=20)
    ax.set_xlabel('w')
    ax.set_ylabel('J(w)')
    ax.set_title(f"Iteration 0")
    
def update(n):
    global wt
    wt = wt - mu*gradJ(wt) # Regla de gradiente descedente
    init()
    ax.set_title(f"Iteration {n+1}")

anim = animation.FuncAnimation(fig, update, init_func=init, frames=15, 
                               interval=1000, repeat=False, blit=False)

In [ ]:
HTML(anim.to_html5_video())

<div class="alert alert-info">
    
Dependiendo de donde partimos la solución final es distinta. El gradiente descedente puede quedarse "atorado" en un mínimo local o en un punto silla
    
</div>

Ahora observe como evolucionan las tres soluciones con $\mu=0.5$, es decir 10 veces más grande que el caso anterior

In [ ]:
%%capture 

mu = 0.5 # Tasa de aprendizaje
wt = np.array([0.05, 0.4, 1.9]) # Solución inicial

fig, ax = plt.subplots(1, figsize=(7, 3), tight_layout=True)
anim = animation.FuncAnimation(fig, update, init_func=init, frames=15, 
                               interval=1000, repeat=False, blit=False)

In [ ]:
HTML(anim.to_html5_video())

<div class="alert alert-info">
    
Si la tasa de aprendizaje es muy alta, los pasos son muy largos y podríamos no converger a un punto estacionario
    
</div>

### Gradiente descendente en el filtro de Wiener

Para el filtro de Wiener teniamos

$$
J(h) = \sigma_d^2 - 2 \textbf{h}^T R_{ud} + \textbf{h}^T R_{uu} \textbf{h},
$$

por ende

$$
\frac{dJ(h)}{dh} = -2 R_{ud} + 2 R_{uu} \textbf{h}
$$

y finalmente

$$
\textbf{h}_{t+1} = \textbf{h}_{t} (I - 2 \mu R_{uu}) + 2\mu R_{ud}
$$

En este caso la condición para una convergencia estable es 

$$
0 < \mu < \frac{1}{\lambda_{\text{max}}},
$$

donde $\lambda_{\text{max}}$ es valor propio más grande de $R_{uu}$ 

(La prueba de esto puede encontrarse en *Haykin, "Adaptive filter theory", Sección 4.3*)

## Gradiente descendente estocástico (SGD)

El filtro de Wiener es óptimo pero no adaptivo:

- Requiere de $N$ muestras de $u$ y $d$ para estimar $R_{ud}$ y $R_{uu}$
- Los pesos se adaptan luego de haber presentado las $N$ muestras: Es una estrategia de tipo **batch**
- Asume que la señal es estacionaria


Consideremos el caso en que los datos no son estacionarios

- Significa que debemos adaptar el filtro en cada paso a medida que nuevas muestras son observadas
- Para lograr esto podemos usar la versión estocástica del GD: SGD
- En SGD los pesos se adaptan luego de haber presentado una sola muestra o un conjunto pequeño de muestras (mini-batch)
- En SGD no hay garantía de llegar al óptimo en un problema convexo, pero es más eficiente computacionalmente que GD

El siguiente esquema muestra una comparación entre la trayectoria de $w$ cuando se usa GD (negro) y SGD (rojo)

En general la trayectoria de SGD será más ruidosa y también podría requerir más pasos, pero cada paso es mucho más económico

<img src="../images/adaptive-sgd.png" width="600">

## Algoritmo Least Mean Square (LMS)


Podemos extender el filtro de Wiener al caso no-estacionario usando SGD, el resultado es un algoritmo simple que además es robusto: El algoritmo LMS

- A diferencia del filtro de Wiener no se requiere conocimiento estadístico del proceso
- Tampoco se requiere calcular e invertir la matriz de correlación
- Se entrena de manera recursiva y online

Consideremos la función de costo **estocástica** para la arquitectura FIR

$$
\begin{align}
J^s_n(\textbf{w}) &= e_n^2 \nonumber \\
&= (d_n - y_n)^2 \nonumber \\
&= (d_n - \textbf{w}^T \textbf{u}_n )^2 \nonumber \\
&= (d_n - \sum_{k=0}^{L} w_{n, k} u_{n-k} )^2 \nonumber 
\end{align}
$$

donde definimos $\textbf{u}_n = [u_n, u_{n-1}, \ldots, u_{n-L}]$. 

Notemos que a diferencia del filtro de Wiener no aplicamos el valor esperado al error cuadrático. Se usa el error cuadrático instantaneo



Para continuar calculamos el gradiente en función del peso $w_{n, k}$ 

$$
\frac{d J^s_n (\textbf{w})}{d w_{n, k}} = - 2 e_n u_{n-k}
$$

Luego, usando la regla SGD llegamos a 

$$
w_{n+1, k} = w_{n, k} + 2 \mu e_n u_{n-k}, k=0, 1, \ldots, L
$$

y que en forma matricial es

$$
\begin{align}
\textbf{w}_{n+1} &= \textbf{w}_{n} + 2 \mu e_n \textbf{u}_{n}\nonumber \\
&= \textbf{w}_{n} + 2 \mu (d_n -  \textbf{w}_{n}^T \textbf{u}_{n}) \textbf{u}_{n}, \nonumber 
\end{align}
$$

que se conoce como la regla de Widrow-Hoff

<div class="alert alert-info">
    
En resumen: Estimamos el error instantaneo y actualizamos los pesos recursivamente
    
</div>

La complejidad de este algoritmo es $L+1$, es decir la complejidad del filtro

Este algoritmo fue inventado en 1960 por [Bernard Widrow](https://en.wikipedia.org/wiki/Bernard_Widrow) y Ted Hoff

### Convergencia del algoritmo LMS (Haykin 6.5)

El algoritmo LMS tiende en la media al valor óptimo

$$
\mathbb{E}[\textbf{w}_n] \to \textbf{w}^*
$$ 

para $n\to \infty$

Además convergence en la media cuadrada: La varianza de $\textbf{w}_n - \textbf{w}^*$ tiene al valor mínimo de $J$ para $n\to \infty$

Esto se cumple si 

$$
0 < \mu < \frac{2}{\text{Tr}[R_{uu}]}
$$

donde $R_{uu} = \mathbb{E}[\textbf{u}_n \textbf{u}_n^T ]$ es la matriz de autocorrelación y $\text{Tr}[]$ el operador que calcula la traza de una matriz

### Algoritmo Normalized LMS (NLMS)

Tenemos la siguiente regla iterativa

$$
\begin{align}
\textbf{w}_{n+1} &= \textbf{w}_{n} + 2 \mu (d_n -  \textbf{w}_{n}^T \textbf{u}_{n}) \textbf{u}_{n} \nonumber \\
& = \textbf{w}_{n} + \Delta \textbf{w}_n \nonumber
\end{align}
$$

que se puede interpretar graficamente como

<img src="../images/adaptive-lms-geometry.png" width="400">

(donde $\textbf{x}(k)$ y $\textbf{w}(k)$ corresponden a $\textbf{u}_n$ y $\textbf{w}_n$ en nuestra notación, respectivamente)

Notemos que

- Los cambios en el vector de peso $\Delta \textbf{w}_n$ son paralelos a $\textbf{u}_{n}$
- Estos cambios podrían estar dominados por 

$$
\max \textbf{u}_{n} = [u_n, u_{n-1}, \ldots, u_{n-L}]
$$

- El algoritmo Normalized LMS (NLMS) corrige este problema ponderando por la varianza de $\textbf{u}_{n}$ 

$$
\textbf{w}_{n+1} = \textbf{w}_{n} + 2 \mu (d_n -  \textbf{w}_{n}^T \textbf{u}_{n}) \frac{\textbf{u}_{n}}{\left(\|\textbf{u}_{n}\|^2 + \delta\right)}
$$

donde la constante $\delta$ es un valor pequeño que se usa para evitar divisiones por cero


En lo que sigue usaremos NLMS para revisar algunas aplicaciones

## Implementación del filtro NLMS y ejemplos de aplicaciones


Podemos implementar las ecuaciones del filtro NLMS como se muestra a continuación

El filtro recibe como entrada el orden $L$ y la tasa de aprendizaje $\mu$

Se asume un vector cero para los pesos iniciales, pero también en la práctica podemos partir de una solución anterior si esta existiera 

Para actualizar el vector de pesos es necesario entregar el vector $\textbf{u}_n \in \mathbb{R}^{L+1}$ y la salida deseada $d_n \in \mathbb{R}$. Luego de actualizar se retorna el valor de la salida predicha por el filtro 

$$
y_n = w_n^T \textbf{u}_n
$$

Estudie el código y compare con las ecuaciones presentadas anteriormente

In [ ]:
class Filtro_NLMS:
    
    def __init__(self, L, mu, delta=1e-6, winit=None):
        self.L = L
        self.w = np.zeros(shape=(L+1, ))
        self.mu = mu
        self.delta = delta
        
    def update(self, un, dn):
        # Asumiendo que un = [u[n], u[n-1], ..., u[n-L]]
        unorm = np.dot(un, un) + self.delta
        yn = np.dot(self.w, un)
        self.w += 2*self.mu*(dn - yn)*(un/unorm)
        return yn

A continuación probaremos este filtro con una aplicación conocida como **Adaptive line enhancement** (ALE).

ALE se refiere a un sistema adaptivo para eliminar ruido blanco aditivo de una señal. El sistema aprende un filtro pasabanda en torno a la frecuencia de interés

En ALE usamos como señal deseada 

$$
d_n = u_n = \textbf{u}_n[0]
$$

El valor predicho por el filtro será la señal $u$ pero libre de ruido blanco. Esto se debe a que el ruido blanco no tiene correlación y por ende no el filtro adaptivo no lo puede predecir

In [ ]:
# Digamos que u = s + n
# El objetivo es limpiar u para obtener s
# s es una señal determínista y n es ruido blanco

Fs, f0 =  50, 5
t = np.arange(0, 4, 1/Fs)
s = np.sin(2.0*np.pi*t*f0)
n = 0.5*np.random.randn(len(t)) 
s[t>2.0] += 5  # Simulemos un cambio abrupto en la media de la señal
#s += s*(0.5 + 0.5*np.cos(2.0*np.pi*t/2))  # Tremolo (AM)
u = s + n

A diferencia de un filtro estático (como el filtro de Wiener) es posible filtrar incluso ante cambios bruscos en la señal. 

Veamos como cambia el resultado del filtro con distintos valores de $\mu$

In [ ]:
from bokeh.palettes import Dark2_5 as palette

L = 20
mus =  np.logspace(-2, 0, num=5)

u_pred = np.zeros(shape=(len(u), len(mus)))
for i, mu in enumerate(mus):
    myfilter = Filtro_NLMS(L=L, mu=mu)
    for k in range(L+1, len(u)):
        u_pred[k, i] = myfilter.update(u[k-L-1:k][::-1], u[k])

p = [Figure(plot_width=620, plot_height=230, toolbar_location="below") for mu in mus]
for i, mu in enumerate(mus):
    p[i].line(t, s, color='green', line_width=2, legend_label=f"Señal limpia");  
    p[i].scatter(t, u, color='black', legend_label=f"Señal ruidosa")
    p[i].line(t, u_pred[:, i], color='blue', 
           line_width=2, legend_label=f"Señal filtrada"); 
    p[i].title.text = f"Tasa de aprendizaje: {mu:0.4f}"
    p[i].legend.location = "top_left"
    
show(column(p))


<div class="alert alert-info">
    
La tasa de aprendizaje $\mu$ controla la velocidad de adaptación. Pero una tasa demasiado grande provoca que el filtro sea inestable. 
    
En general el valor óptimo de $\mu$ depende del problema y del valor de $L$
    
</div>

In [ ]:
L = 20
myfilter = Filtro_NLMS(L=20, mu=0.02)
w = np.zeros()
for k in range(L+1, len(u)):
    u_pred[k, i] = myfilter.update(u[k-L-1:k][::-1], u[k])
        
fk, Hk = scipy.signal.freqz(b=myfilter.w, a=1, fs=Fs)

## Resumen de comparación entre Filtro de Wiener/GD y algoritmo LMS/SGD

- **Supuestos**:  Wiener requiere un ambiente estacionario lo cual nos permite calcular $R_{uu}$ y $R_{ud}$. En LMS la señal puede ser no estacionaria.
- **Aprendizaje:** En el filtro de Wiener el aprendizaje es determinista. En LMS el aprendizaje viene **promediando** a nivel de los estimadores de $w$. En LMS el aprendizaje es estadístico. 
- **Optimalidad:** Wiener es óptimo en cambio LMS es sub-óptimo (localmente óptimo). LMS tiende a la solución de Wiener
- **Costo:** LMS se actualiza online y tiene costo $L$. Wiener se entrena offline y tiene costo $L^2$

Diagrama que compara el filtro de Wiener y el algoritmo LMS






<img src="../images/adaptive-lms.png">